<a href="https://colab.research.google.com/github/kanupriya1125/sentiments/blob/main/sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
import numpy as np
import datetime
import glob
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras 
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

from PIL import Image
from IPython.display import display
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

In [4]:
root_dir ="/content/drive/MyDrive/tweet-sentiment/"

In [5]:
df=pd.read_csv(root_dir+"train.csv")

In [6]:
df.isna().sum()

textID           0
text             1
selected_text    1
sentiment        0
dtype: int64

In [7]:
df = df.dropna()

In [8]:
df["sentiment"].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [9]:
df[["sentiment"]] = df[["sentiment"]].replace(to_replace =["neutral"], 
                            value =0)
df[["sentiment"]] = df[["sentiment"]].replace(to_replace =["negative"], 
                            value =1)
df[["sentiment"]] = df[["sentiment"]].replace(to_replace =["positive"], 
                            value =2)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(df['text'],df['sentiment'],test_size=0.2, stratify=df['sentiment'],random_state=0)

In [11]:
y_train = np.array(y_train)
y_test = np.array(y_test)
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 2000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x_train)

In [13]:
train_sequences = tok.texts_to_sequences(x_train)
train_sequences = sequence.pad_sequences(train_sequences,maxlen=150)

In [14]:
test_sequences = tok.texts_to_sequences(x_test)
test_sequences = sequence.pad_sequences(test_sequences,maxlen=150)

In [15]:
import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras import Sequential
from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras import regularizers
from tensorflow.keras.applications import VGG16,VGG19


def build_model(NUM_CLASSES):

    input =  tf.keras.layers.Input(name='input',shape=[150])

   


 

    
    max_words = 2000
    max_len = 200
    text = L.Embedding(max_words,100,input_length=max_len)(input)
    text = L.BatchNormalization()(text)
    text = L.LSTM(500,activation="relu")(text)


    
    # text = L.LSTM(128,return_sequences = True,activation="relu")(text)
    # text = L.Flatten()(text)
    text = L.Dense(units=512,activation="relu")(text)
    
    
    
    outputs = L.Dense(NUM_CLASSES, activation="softmax", )(text)
    
    




    # Compile
    model = Model(inputs=input, outputs=outputs, name="SentimentClassifier")


    return model

model = build_model(NUM_CLASSES=3)

In [16]:
model.compile(  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0001
),
  metrics=['accuracy'])

In [17]:
model.summary()

Model: "SentimentClassifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 100)          200000    
                                                                 
 batch_normalization (BatchN  (None, 150, 100)         400       
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 500)               1202000   
                                                                 
 dense (Dense)               (None, 512)               256512    
                                                                 
 dense_1 (Dense)             (None, 3)                 1539      
                                               

In [ ]:
model.fit(
         train_sequences, y_train, 
          validation_data=(test_sequences,y_test),
          epochs=15,
          batch_size=128,
          #  callbacks=callbacks
          )

Epoch 1/15
 57/172 [========>.....................] - ETA: 6:03 - loss: 1.0778 - accuracy: 0.4183

In [ ]:
y_true = np.argmax(y_test,axis=1)
y_true

In [ ]:
y_pred = model.predict(test_sequences,batch_size = 32,verbose=1)
y_pred_1 = np.argmax(y_pred,axis=1)
y_pred_1

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
mat = confusion_matrix(y_true, y_pred_1).astype('float')
mat = (mat/mat.astype(np.float).sum(axis=0))*100

In [ ]:
df_cm = pd.DataFrame(mat, range(3), range(3))
plt.figure(figsize=(10,7))
sns.set(font_scale=1) # for label size
sns.heatmap(df_cm, annot=True, annot_kws={"size": 12},fmt='.1f') # font size

plt.show()